In [1]:
import sys
sys.path.append("..")
sys.path.append("../src")

import config
from dataIO import load_data
from vectorDB import QdrantManager
from vectorDB import PaperEmbed

In [2]:
embedEngine = PaperEmbed(model_name=config.MODEL_EMBED)
qdrant = QdrantManager()


2025-04-11 18:22:42,699 - vectorDB.push_qdrant - INFO - Qdrant container already up and running


In [4]:
query = "what's the role of the basolateral amygdala?"

In [5]:
query_emb = embedEngine.embed_queries([query])

In [6]:
import nest_asyncio
nest_asyncio.apply()

res = await qdrant.query_batch_streamed(query_vectors=query_emb.tolist())
match_titles = [res[0].groups[k].lookup.payload['title'] for k in range(len(res[0].groups))]
match_texts = [res[0].groups[k].lookup.payload['main_text'] for k in range(len(res[0].groups))]

In [7]:
from transformers import pipeline
import torch

# model_id = "unsloth/Mistral-Small-24B-Instruct-2501-unsloth-bnb-4bit"
model_id = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit"
pipe = pipeline(
    "text-generation",
    model=model_id)

`low_cpu_mem_usage` was None, now default to True since model is quantized.
Device set to use cuda:0


In [10]:
articles = '\n\nabstract#'.join(f"{k}\n" + txt for k, txt in enumerate(match_texts[2:3]))
query="What is the role of amygdala?"
messages = [
    {"role": "system", "content": """
Summarize the article below to help answer a question. 
Do not directly answer the question, instead summarize the evidence present in the excerpt that
help answer the question. 
Reply ’Not applicable’ if the excerpt is irrelevant. 
At the end of your response, provide a score from 1-10 on a newline indicating the relevance
of the text to the question. 
DO NOT EXPLAIN YOUR SCORE.
     """},
    {"role": "user", "content": f"""
Question: {query}
Article: 
{articles}

Relevant Information Summary:"""}
]
# query="What is the role of amygdala?"
# messages = f"""<s>[SYSTEM_PROMPT]Summarize the article below to help answer a question. 
# Do not directly answer the question, instead summarize the evidence present in the excerpt that
# help answer the question. 
# Reply ’Not applicable’ if the excerpt is irrelevant. 
# At the end of your response, provide a score from 1-10 on a newline indicating the relevance
# of the text to the question. 
# Do not explain your score.[/SYSTEM_PROMPT]

# [INST]
# Question: {query}
# Article: 
# {articles}

# Relevant Information Summary:[/INST]"""

outputs = pipe(
    messages,
    max_new_tokens=10000,
)
# print(outputs[0]["generated_text"][-1]['content'])

In [11]:
print(outputs[0]["generated_text"][-1]['content'])

 The article discusses the role of dynorphin-expressing neurons in the central amygdala (CeADyn) in alcohol drinking. The study used fiber photometry to record real-time activity of these neurons in male and female mice during voluntary alcohol consumption, water consumption, and sucrose consumption. The results showed that there is a unique functional signature for alcohol in the CeADyn neuron population, with a large increase in calcium transients after bouts of licking for alcohol, and only modest increases during licking for water or 0.5% sucrose. This increase in activity was present even when controlling for differences in drinking behavior unique to alcohol, such as longer bout durations. The study suggests that understanding the specific brain mechanisms underlying the behavioral and physiological responses to alcohol that promote alcohol misuse is important for understanding the role of these neurons in alcohol drinking.

Relevance Score: 10

Explanation: The article provides 

In [37]:
print(outputs[0]["generated_text"].split("[/INST]")[-1])

The article discusses a study that investigates the activity of dynorphin-expressing neurons in the central amygdala (CeADyn) during alcohol drinking in mice. Key findings include:

- CeADyn neurons show a large increase in calcium transients after bouts of licking for alcohol, compared to modest increases for water or sucrose.
- The unique engagement of CeADyn neurons during alcohol consumption is not fully explained by differences in drinking behavior, such as longer bout durations for alcohol.
- The study used fiber photometry to record neuronal activity and multilevel modeling to analyze the data, revealing significant effects of time and solution type on neuronal activity.
- The study also explored the relationship between drinking characteristics (such as bout duration and temporal occurrence) and neuronal activity, finding that while these characteristics modulate signal, they do not fully explain the unique alcohol-evoked CeADyn activity.
- The study suggests that the unique ph

In [19]:
print(messages)

<s>[SYSTEM_PROMPT]You are a research assistant. Answer the user’s query as accurately as possible using your knowledge and the provided articles.     Do not summarize the abstracts unless explicitly asked. Use your knowledge first if the question is general. Try as much as possible to cite the provided abstract which are      referenced as abstract#1, abstract#2, etc[/SYSTEM_PROMPT]     [INST]Here are the abstracts of recent papers:

0
AmygdalaGo-BOLT3D: A boundary learning transformer for tracing human amygdala 1Center for Brain Imaging Science and Technology, Zhejiang University, Hangzhou, 310027, Zhejiang, China. 2College of Biomedical Engineering and Instrument Science, Zhejiang University, Hangzhou, 310027, China. 3State Key Laboratory of Cognitive Neuroscience and Learning, Beijing Normal University, Beijing, 100875, China. 4Faculty of Psychology, Beijing Normal University, Beijing, 100875, Beijing, China. 5School of Psychology, Capital Normal University, Beijing, 100190, China. 